安装MindSpore框架和MindNLP套件

In [3]:
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.4.1/MindSpore/unified/aarch64/mindspore-2.4.1-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.5/335.5 MB 6.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.4/442.4 kB 8.2 MB/s eta 0:00:00a 0:00:01
DEPRECATION: moxing-framework 2.1.16.2ae09d45 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of moxing-framework or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: mindspore
    Found existing installation: mindspore 2.3.0
    Uninstalling mindspore-2.3.0:
      Successfully uninstalled mindspore-2.3.0


In [ ]:
!pip install mindnlp

In [1]:
!pip install jieba

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
DEPRECATION: moxing-framework 2.1.16.2ae09d45 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of moxing-framework or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [8]:
%env HF_ENDPOINT=https://hf-mirror.com

env: HF_ENDPOINT=https://hf-mirror.com


In [1]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn
mindspore.set_context(device_target='Ascend', device_id=0)

from mindnlp.dataset import load_dataset

from mindnlp.engine.trainer import Trainer

[WARNING] GE_ADPT(63944,ffffa8aeb010,python):2024-12-23-19:00:28.376.291 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleGetModelId failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleGetModelId
[WARNING] GE_ADPT(63944,ffffa8aeb010,python):2024-12-23-19:00:28.376.360 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleLoadFromMem failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleLoadFromMem
[WARNING] GE_ADPT(63944,ffffa8aeb010,python):2024-12-23-19:00:28.376.379 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleUnload failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleUnload
[WARNING] ME(63944:281473511763984,MainProcess):2024-12-23-19:00:28.511.737 [mindspore/run_check/_check_version.py:398

In [2]:
imdb_ds = load_dataset('imdb', split=['train', 'test'])
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']

In [3]:
imdb_train.get_dataset_size()

25000

In [4]:
import numpy as np

def process_dataset(dataset, tokenizer, max_seq_len=512, batch_size=4, shuffle=False):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'
    def tokenize(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['attention_mask']

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                             'attention_mask': (None, 0)})

    return dataset

In [5]:
from mindnlp.transformers import OpenAIGPTTokenizer
# tokenizer
gpt_tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

# add sepcial token: <PAD>
special_tokens_dict = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
}
num_added_toks = gpt_tokenizer.add_special_tokens(special_tokens_dict)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [6]:
# split train dataset into train and valid datasets
imdb_train, imdb_val = imdb_train.split([0.7, 0.3])

In [7]:
dataset_train = process_dataset(imdb_train, gpt_tokenizer, shuffle=True)
dataset_val = process_dataset(imdb_val, gpt_tokenizer)
dataset_test = process_dataset(imdb_test, gpt_tokenizer)

In [8]:
next(dataset_train.create_tuple_iterator())

[Tensor(shape=[4, 512], dtype=Int64, value=
 [[  246, 40268,   544 ... 40480, 40480, 40480],
  [ 4605,   239,   239 ... 40480, 40480, 40480],
  [  862,   239,   239 ... 40480, 40480, 40480],
  [  249,   604,   246 ...   239,  1096,   513]]),
 Tensor(shape=[4, 512], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 1, 1, 1]]),
 Tensor(shape=[4], dtype=Int32, value= [0, 1, 0, 0])]

In [9]:
import evaluate
import numpy as np
from mindnlp.engine.utils import EvalPrediction

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred: EvalPrediction):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from mindnlp.transformers import OpenAIGPTForSequenceClassification
from mindnlp.engine import TrainingArguments

model = OpenAIGPTForSequenceClassification.from_pretrained('openai-gpt', num_labels=2)
model.config.pad_token_id = gpt_tokenizer.pad_token_id
model.resize_token_embeddings(model.config.vocab_size + 3)

training_args = TrainingArguments(
    "./output/gpt",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=2e-5,
    num_train_epochs=3,
    logging_steps=200,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(model=model, train_dataset=dataset_train,
                  eval_dataset=dataset_val, compute_metrics=compute_metrics,
                  args=training_args)

[WARNING] DEVICE(63944,ffffa8aeb010,python):2024-12-23-19:01:48.707.982 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:188] CheckVmmDriverVersion] Driver version is less than 24.0.0, vmm is disabled by default, drvier_version: 23.0.rc2.2
Some weights of OpenAIGPTForSequenceClassification were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 注: 如果想要运行的更快一些，可以在训练时需要V100的算力

In [22]:
# trainer.run(tgt_columns="labels")
trainer.train()

  0%|          | 0/13125 [00:00<?, ?it/s]

  0%|          | 1/13125 [00:34<126:27:08, 34.69s/it]

  2%|▏         | 200/13125 [02:32<1:11:24,  3.02it/s]

{'loss': 0.6092, 'learning_rate': 1.9695238095238096e-05, 'epoch': 0.05}


  3%|▎         | 401/13125 [03:44<1:12:35,  2.92it/s]

...

100%|█████████▉| 1870/1875 [02:22<00:00, 26.48it/s]
                                                       
100%|██████████| 1875/1875 [02:22<00:00, 26.51it/s]
                                                   

{'eval_loss': 0.46615397930145264, 'eval_accuracy': 0.9292, 'eval_runtime': 143.8717, 'eval_samples_per_second': 13.032, 'eval_steps_per_second': 13.032, 'epoch': 3.0}


100%|██████████| 13125/13125 [1:26:42<00:00,  2.52it/s]

{'train_runtime': 5202.182, 'train_samples_per_second': 10.092, 'train_steps_per_second': 2.523, 'train_loss': 0.21964950626918248, 'epoch': 3.0}


TrainOutput(global_step=13125, training_loss=0.21964950626918248, metrics={'train_runtime': 5202.182, 'train_samples_per_second': 10.092, 'train_steps_per_second': 2.523, 'train_loss': 0.21964950626918248, 'epoch': 3.0})

In [23]:
# evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
# evaluator.run(tgt_columns="labels")
from tqdm import tqdm

def compute_accuracy(logits, labels):
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def evaluate_fn(model, test_dataset):
    total = test_dataset.get_dataset_size()
    epoch_acc = 0
    step_total = 0
    model.set_train(False)

    with tqdm(total=total) as progress_bar:
        for batch in test_dataset.create_dict_iterator():
            label = batch.pop('labels')
            logits = model(**batch).logits

            acc = compute_accuracy(logits, label)['accuracy']
            epoch_acc += acc
            
            step_total += 1
            acc=epoch_acc/step_total

    return acc

In [24]:
acc = evaluate_fn(model, dataset_val)
print(f"Accuracy: {acc}")

  0%|          | 0/1875 [02:40<?, ?it/s]

Accuracy: 0.9292
